## Importing Libraries 

In [1]:
from bs4 import BeautifulSoup
from datetime import datetime,date, timedelta

import urllib.request
import requests
import lxml.html as lh
import pandas as pd
import numpy as np

## Web Scraping from Epex Website

### 1. Accessing the Web Page

In [2]:
def date_selection(x):
    url1 = 'https://www.epexspot.com/en/market-data?market_area=DE&trading_date='
    url2 = '&delivery_date=' + x
    url3 = '&underlying_year=&modality=Continuous&sub_modality=&product=60&data_mode=table&period='
    url = url1 + url2 + url3  
    return(url)

In [3]:
### NOTE: There is no data available for intraday continuous fot the next day i.e. tomorrow.

date_download = input("Enter today, yesterday or tomorrow: ")

Enter today, yesterday or tomorrow: today


In [4]:
if date_download == 'today':
    date_download1 = str(date.today())
elif date_download == 'yesterday':
    date_download1 = str(date.today() - timedelta(days = 1))
elif date_download == 'tomorrow':
    date_download1 = str(date.today() + timedelta(days= 1))
else:
    data_download

In [5]:
#url = 'https://www.epexspot.com/en/market-data?market_area=DE&trading_date=&delivery_date=2019-12-14&underlying_year=&modality=Continuous&sub_modality=&product=60&data_mode=table&period='

In [6]:
url = date_selection(date_download1)
print(url)

https://www.epexspot.com/en/market-data?market_area=DE&trading_date=&delivery_date=2020-01-05&underlying_year=&modality=Continuous&sub_modality=&product=60&data_mode=table&period=


In [7]:
#Each browser has a unique way of communicating with the website. 
#Headers tells the website which browser is being used and what your operating system is 
#Header obtained from www.whoishostingthis.com/tools/user-agent/

headers = {'User-Agent' : 'Chrome/78.0.3904.108 '}

# fetch the content from url

page = requests.get(url, timeout=5, headers = headers)

#To check your connection to the website and to check if the html content has been downloaded
#200 indicates good connection and proper download

print(page.status_code)

200


### 2. Retrieving the Data

In [8]:
#Parsing the HTML content

soup = BeautifulSoup(page.content, 'html.parser')
print(soup.prettify())

<!DOCTYPE html>
<html dir="ltr" lang="en" prefix="content: http://purl.org/rss/1.0/modules/content/  dc: http://purl.org/dc/terms/  foaf: http://xmlns.com/foaf/0.1/  og: http://ogp.me/ns#  rdfs: http://www.w3.org/2000/01/rdf-schema#  schema: http://schema.org/  sioc: http://rdfs.org/sioc/ns#  sioct: http://rdfs.org/sioc/types#  skos: http://www.w3.org/2004/02/skos/core#  xsd: http://www.w3.org/2001/XMLSchema# ">
 <head>
  <meta charset="utf-8"/>
  <meta content="Market Data | EPEX SPOT" name="title"/>
  <link href="https://www.epexspot.com/en/market-data" rel="canonical"/>
  <meta content="width" name="MobileOptimized"/>
  <meta content="true" name="HandheldFriendly"/>
  <meta content="width=device-width, initial-scale=1.0" name="viewport"/>
  <script>
   function euCookieComplianceLoadScripts() {}
  </script>
  <style>
   div#sliding-popup, div#sliding-popup .eu-cookie-withdraw-banner, .eu-cookie-withdraw-tab {background: #0779bf} div#sliding-popup.eu-cookie-withdraw-wrapper { backgro

In [9]:
#Extracting the table

#"table" is the tag while class is the attribute always distinguished by an '_'
epex_table = soup.find_all("table", class_= "table-01")

#Checking how many tables have been downloaded (class is unique)
print(len(epex_table))

#Gets a portion of the HTML content which was previously stored in soup (Extracts the only result)
epex_table = epex_table[0]

#Check for its type
print(type(epex_table))


1
<class 'bs4.element.Tag'>


In [10]:
#Extracting the column names

epex_cols = []

for row in epex_table.find_all('tr'):
    for cell in row.find_all('th'):
        epex_cols.append(cell.text.strip())
        
del epex_cols[0]
print(epex_cols)

['Low(€/MWh)', 'High(€/MWh)', 'Last(€/MWh)', 'Weight Avg.(€/MWh)', 'Buy Volume(MWh)', 'Sell Volume(MWh)', 'Volume(MWh)']


In [11]:
#Extracting each element of each row in the epex_table

table = []

for row in epex_table.find_all('tr'):
    r = []
    for value in row.find_all('td'):
        r.append(value.text)
    table.append(r)

del table[0:2]

table_cols = pd.DataFrame(table, columns = epex_cols)
table_cols.head()

,Low(€/MWh),High(€/MWh),Last(€/MWh),Weight Avg.(€/MWh),Buy Volume(MWh),Sell Volume(MWh),Volume(MWh)
0,16.09,35.45,26.00,30.24,"3,117.5","2,670.9","2,894.2"
1,-,-,-,-,-,-,-
2,-21.99,40.09,-21.99,23.08,337.0,311.5,324.2
3,10.00,31.19,18.90,24.95,198.4,195.9,197.1
4,-,-,-,-,-,-,-


In [13]:
#Extracting the hours column from the HTML content (for this website it is embedded in the <a href> tag)

hours_text = []
for a in soup.find_all('a'): 
     hours_text.append(a.text)
        
#Obtain the index of the required elements
i_1 = hours_text.index('00 - 01')
print(hours_text.index('00 - 01'))

i_2 = hours_text.index('23:45 - 24:00')
print(hours_text.index('23:45 - 24:00'))

i_end = i_2 + 1

#Using their indices, extract these elements
hours = hours_text[i_1: i_end]

#Add hours list to the table_cols dataframe
table_cols['Hours'] = hours

#Rearrange the columns
table_cols = table_cols[['Hours', 'Low(€/MWh)', 'High(€/MWh)', 'Last(€/MWh)', 'Weight Avg.(€/MWh)', 'Buy Volume(MWh)', 'Sell Volume(MWh)', 'Volume(MWh)' ]]

#View the result
table_cols.head()

57
224


,Hours,Low(€/MWh),High(€/MWh),Last(€/MWh),Weight Avg.(€/MWh),Buy Volume(MWh),Sell Volume(MWh),Volume(MWh)
0,00 - 01,16.09,35.45,26.00,30.24,"3,117.5","2,670.9","2,894.2"
1,00:00 - 00:30,-,-,-,-,-,-,-
2,00:00 - 00:15,-21.99,40.09,-21.99,23.08,337.0,311.5,324.2
3,00:15 - 00:30,10.00,31.19,18.90,24.95,198.4,195.9,197.1
4,00:30 - 01:00,-,-,-,-,-,-,-
